# Analysis of Mesoscale Eddies in the Black Sea
**Author:** Rahan Öztürk  
**Dataset:** Black Sea Mesoscale Eddy Dataset (1995–2019)

This notebook demonstrates how to:
1. Load and explore the climatological dataset.
2. Visualize the spatial distribution of different eddy clusters.
3. Compare Warm-Core eddies against the general population.
4. Demonstrate the **Spiral Template-Matching Algorithm** using synthetic data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import sys
import os

# Add 'src' to path to import local modules
sys.path.append(os.path.abspath("src"))

from spiral_algorithm import cyclone_score, wind_dir_deg, get_template

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')


In [ ]:
# Load the dataset
df = pd.read_csv('data/black_sea_mesoscale_eddies_1995_2019.csv')

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

print(f"Total Records: {len(df)}")
print(f"Unique Tracks: {df['track_id_global'].nunique()}")
print("-" * 30)
df.info()


In [ ]:
# Cluster Distribution
# 1: Caucasus, 2: Caucasus-SE, 3: Küre, 4: Küre-West, 5: Crimea
cluster_counts = df.groupby('cluster')['track_id_global'].nunique()
print("Unique Eddies per Cluster:")
print(cluster_counts)

print("\nWarm-Core vs Standard Eddies:")
print(df['is_warm_core'].value_counts().rename({0: 'Standard', 1: 'Warm-Core'}))


In [ ]:
# Visualize Genesis Locations by Cluster
# We will plot the first point of each unique track

genesis_df = df.sort_values('date').groupby('track_id_global').first()

fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([27.0, 42.0, 40.5, 47.0], crs=ccrs.PlateCarree())

# Features
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Colors for clusters
colors = {1: 'red', 2: 'orange', 3: 'purple', 4: 'blue', 5: 'green'}
labels = {1: 'Caucasus', 2: 'Caucasus-SE', 3: 'Küre', 4: 'Küre-West', 5: 'Crimea'}

# Plot each cluster
for cls_id, color in colors.items():
    subset = genesis_df[genesis_df['cluster'] == cls_id]
    ax.scatter(subset['lon'], subset['lat'], c=color, s=10, 
               label=labels[cls_id], alpha=0.6, transform=ccrs.PlateCarree())

# Plot Non-clustered (NaN) in gray
subset_nan = genesis_df[genesis_df['cluster'].isna()]
ax.scatter(subset_nan['lon'], subset_nan['lat'], c='gray', s=5, 
           label='Open Sea / Other', alpha=0.3, transform=ccrs.PlateCarree())

plt.title('Genesis Locations of Black Sea Mesoscale Eddies (1995-2019)', fontsize=14)
plt.legend(loc='lower left', frameon=True)
plt.show()


In [ ]:
# Compare Warm-Core vs Standard Eddies
# Warm-core eddies are expected to have higher thermal contrast (dT_ring) 
# and often higher wind speeds.

plt.figure(figsize=(10, 6))

sns.scatterplot(data=df, x='max_speed', y='dT_ring', 
                hue='is_warm_core', style='is_warm_core',
                palette={0: 'gray', 1: 'red'}, alpha=0.6)

plt.axhline(1.0, color='k', linestyle='--', linewidth=0.8, label='Warm Core Threshold (1K)')
plt.axvline(18.0, color='blue', linestyle='--', linewidth=0.8, label='Wind Threshold (18m/s)')

plt.title('Dynamic vs. Thermodynamic Characteristics')
plt.xlabel('Max Wind Speed (m/s)')
plt.ylabel('Core-Ring Thermal Contrast (K)')
plt.legend(title='Eddy Type')
plt.show()


## Algorithm Demonstration: Synthetic Vortex

Since the raw COSMO-REA6 NetCDF files are too large to host in this repository, we demonstrate the **Spiral Template-Matching Algorithm** using a synthetically generated wind field.

We will:
1. Create a synthetic cyclone (Rankine vortex-like structure).
2. Calculate the wind direction.
3. Apply the `cyclone_score` function from `src/spiral_algorithm.py`.
4. Verify that the algorithm identifies the center (Score minimum).


In [ ]:
# 1. Create a grid
x = np.linspace(-100, 100, 41) # km
y = np.linspace(-100, 100, 41) # km
X, Y = np.meshgrid(x, y)

# 2. Create a Synthetic Cyclonic Vortex (Counter-Clockwise)
# V_theta = r * exp(-r^2) type simple profile
R = np.sqrt(X**2 + Y**2)
V_tangential = 20 * (R / 25) * np.exp(-(R/25)**2)  # Peak winds at R=25km

# Convert to U, V components
# u = -V_t * sin(theta), v = V_t * cos(theta)
Theta = np.arctan2(Y, X)
U = -V_tangential * np.sin(Theta)
V = V_tangential * np.cos(Theta)

# 3. Calculate Wind Direction (Meteorological)
# Using the function imported from src/
wd_grid = wind_dir_deg(U, V)

# 4. Apply Spiral Algorithm
# We use a dummy sea_mask (all sea = 1)
sea_mask = np.ones_like(wd_grid)

# Calculate Score
# n=3 corresponds to a small window for this coarse synthetic grid
S = cyclone_score(wd_grid, sea_mask=sea_mask, n=3, tol=45)

# Find minimum score location
min_idx = np.unravel_index(np.nanargmin(S), S.shape)
min_score = S[min_idx]

print(f"Minimum Score Found: {min_score:.2f} at grid index {min_idx}")


In [ ]:
# Visualization of the Algorithm Test
fig, ax = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Wind Field (Quiver)
skip = 2
ax[0].set_title("Synthetic Vortex Wind Field")
ax[0].quiver(X[::skip, ::skip], Y[::skip, ::skip], 
             U[::skip, ::skip], V[::skip, ::skip], scale=400)
ax[0].set_aspect('equal')

# Plot 2: Spiral Score Map
ax[1].set_title("Spiral Mismatch Score (Lower is Better)")
cf = ax[1].contourf(X, Y, S, levels=20, cmap='jet')
plt.colorbar(cf, ax=ax[1], label='Mismatch Score')

# Mark the detected center
ax[1].plot(x[min_idx[1]], y[min_idx[0]], 'ro', label='Detected Center')
ax[1].legend()
ax[1].set_aspect('equal')

plt.tight_layout()
plt.show()
